In [1]:
# Importing data and models

In [27]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib

In [28]:
last_team_scores = pd.read_csv('../Datasets/last_team_scores.csv')
last_team_scores.tail()

,team,date,rank,goalkeeper_score,defense_score,offense_score,midfield_score
27,Switzerland,2022-06-12,14,85.0,78.0,77.0,80.0
28,Tunisia,2022-06-14,35,64.0,71.0,72.0,74.0
29,USA,2022-06-14,15,77.0,76.0,78.0,76.0
30,Uruguay,2022-06-11,13,80.0,81.0,84.0,80.0
31,Wales,2022-06-14,18,74.0,75.0,73.0,78.0


In [29]:
squad_stats = pd.read_csv('../Datasets/squad_stats.csv')
squad_stats.tail()

,nationality_name,overall,potential
27,Australia,72.55,76.82
28,Tunisia,73.00,76.00
29,Saudi Arabia,71.45,75.27
30,Costa Rica,72.00,74.55
31,IR Iran,70.82,72.36


In [30]:
group_matches = pd.read_csv('../Datasets/Qatar_group_stage.csv')

round_16 = group_matches.iloc[48:56, :]
quarter_finals = group_matches.iloc[56:60, :]
semi_finals = group_matches.iloc[60:62, :]
final = group_matches.iloc[62:63, :]
second_final = group_matches.iloc[63:64, :]
group_matches = group_matches.iloc[:48, :]
group_matches.tail()

,country1,country2,group
43,Costa Rica,Germany,e
44,Ghana,Uruguay,h
45,Korea Republic,Portugal,h
46,Serbia,Switzerland,g
47,Cameroon,Brazil,g


In [31]:
group_matches_semi_Fin = pd.read_csv('../Datasets/Qatar_group_stage -Semi_Finals.csv')
group_matches_semi_Fin

,country1,country2
0,France,Morocco
1,Argentina,Croatia


In [32]:
group_matches_3rd = pd.read_csv('../Datasets/Qatar_group_stage_3rd.csv')
group_matches_3rd

,country1,country2
0,Croatia,Morocco


In [33]:
group_matches_Fin = pd.read_csv('../Datasets/Qatar_group_stage_finals.csv')
group_matches_Fin

,country1,country2
0,Argentina,France


In [34]:
gs_model = joblib.load("../groups_stage_prediction.pkl")

In [35]:
ks_model = joblib.load("../groups_stage_prediction.pkl")

In [36]:
team_group = group_matches.drop(['country2'], axis=1)
team_group = team_group.drop_duplicates().reset_index(drop=True)
team_group = team_group.rename(columns = {"country1":"team"})
team_group.head(7)

,team,group
0,Qatar,a
1,Senegal,a
2,England,b
3,USA,b
4,France,d
5,Denmark,d
6,Mexico,c


In [37]:
# Declaring important functions

In [38]:
def matches(g_matches):
    g_matches.insert(2, 'potential1', g_matches['country1'].map(squad_stats.set_index('nationality_name')['potential']))
    g_matches.insert(3, 'potential2', g_matches['country2'].map(squad_stats.set_index('nationality_name')['potential']))
    g_matches.insert(4, 'rank1', g_matches['country1'].map(last_team_scores.set_index('team')['rank']))
    g_matches.insert(5, 'rank2', g_matches['country2'].map(last_team_scores.set_index('team')['rank']))
    pred_set = []

    for index, row in g_matches.iterrows():
        if row['potential1'] > row['potential2'] and abs(row['potential1'] - row['potential2']) > 2:          
            pred_set.append({'Team1': row['country1'], 'Team2': row['country2']})
        elif row['potential2'] > row['potential1'] and abs(row['potential2'] - row['potential1']) > 2:
            pred_set.append({'Team1': row['country2'], 'Team2': row['country1']})
        else:
            if row['rank1'] > row['rank2']:
                pred_set.append({'Team1': row['country1'], 'Team2': row['country2']})
            else:
                pred_set.append({'Team1': row['country2'], 'Team2': row['country1']})
   
    pred_set = pd.DataFrame(pred_set)
    pred_set.insert(2, 'Team1_FIFA_RANK', pred_set['Team1'].map(last_team_scores.set_index('team')['rank']))
    pred_set.insert(3, 'Team2_FIFA_RANK', pred_set['Team2'].map(last_team_scores.set_index('team')['rank']))
    pred_set.insert(4, 'Team1_Goalkeeper_Score', pred_set['Team1'].map(last_team_scores.set_index('team')['goalkeeper_score']))
    pred_set.insert(5, 'Team2_Goalkeeper_Score', pred_set['Team2'].map(last_team_scores.set_index('team')['goalkeeper_score']))
    pred_set.insert(6, 'Team1_Defense', pred_set['Team1'].map(last_team_scores.set_index('team')['defense_score']))
    pred_set.insert(7, 'Team1_Offense', pred_set['Team1'].map(last_team_scores.set_index('team')['offense_score']))
    pred_set.insert(8, 'Team1_Midfield', pred_set['Team1'].map(last_team_scores.set_index('team')['midfield_score']))
    pred_set.insert(9, 'Team2_Defense', pred_set['Team2'].map(last_team_scores.set_index('team')['defense_score']))
    pred_set.insert(10, 'Team2_Offense', pred_set['Team2'].map(last_team_scores.set_index('team')['offense_score']))
    pred_set.insert(11, 'Team2_Midfield', pred_set['Team2'].map(last_team_scores.set_index('team')['midfield_score']))
    return pred_set

In [39]:
 def print_results(dataset, y_pred, matches, proba):
  results = []
  for i in range(dataset.shape[0]):
      print()
      if y_pred[i] == 2:
          print(matches.iloc[i, 0] + " vs. " + matches.iloc[i, 1] + " => Draw")
          results.append({'result': 'Draw'})
      elif y_pred[i] == 1:
          print(matches.iloc[i, 0] + " vs. " + matches.iloc[i, 1] + " => Winner: " + dataset.iloc[i, 0])
          results.append({'result': dataset.iloc[i, 0]})
      else:
          print(matches.iloc[i, 0] + " vs. " + matches.iloc[i, 1] + " => Winner: " + dataset.iloc[i, 1])
          results.append({'result': dataset.iloc[i, 1]})
      try:
          print('Probability of ' + dataset.iloc[i, 0] + ' winning: ', '%.3f'%(proba[i][1]))
          print('Probability of Draw: ', '%.3f'%(proba[i][2]))
          print('Probability of ' + dataset.iloc[i, 1] + ' winning: ', '%.3f'%(proba[i][0]))
      except:   
          print('Probability of ' + dataset.iloc[i, 1] + ' winning: ', '%.3f'%(proba[i][0]))   
      print("")
  results = pd.DataFrame(results)
  matches = pd.concat([matches.group, results], axis=1)
  return matches

In [40]:
def winner_to_match(round, prev_match):
    round.insert(0, 'c1', round['country1'].map(prev_match.set_index('group')['result']))
    round.insert(1, 'c2', round['country2'].map(prev_match.set_index('group')['result']))
    round = round.drop(['country1','country2'], axis=1)
    round = round.rename(columns={'c1':'country1', 'c2':'country2'}).reset_index(drop=True)
    return round

In [41]:
def prediction_knockout(round):
    dataset_round = matches(round)
    prediction_round = ks_model.predict(dataset_round)
    proba_round = ks_model.predict_proba(dataset_round)
    results_round = print_results(dataset_round, prediction_round, round, proba_round)
    return results_round


In [42]:
def center_str(round):
    spaces = ['',' ','  ','   ','    ','     ',]
    for j in range(2):
        for i in range(round.shape[0]):
            if (13 - len(round.iloc[i, j])) % 2 == 0:
                round.iloc[i, j] = spaces[int((13 - len(round.iloc[i, j])) / 2)] + round.iloc[i, j] + spaces[int((13 - len(round.iloc[i, j])) / 2)]
            else:
                round.iloc[i, j] = spaces[int(((13 - len(round.iloc[i, j])) / 2) - 0.5)] + round.iloc[i, j] + spaces[int(((13 - len(round.iloc[i, j])) / 2) + 0.5)]
    return round

In [43]:
def center2(a):
    spaces = ['',' ','  ','   ','    ','     ','      ','       ','        ','         ','          ','           ','            ','             ','              ','               ','                ','                 ','                  ','                   ','                    ']
    if (29 - len(a)) % 2 == 0:
        a = spaces[int((29 - len(a)) / 2)] + a + spaces[int((29 - len(a)) / 2)]
    else:
        a = spaces[int(((29 - len(a)) / 2) - 0.5)] + a + spaces[int(((29 - len(a)) / 2) + 0.5)]
    return a

In [44]:
# GROUP STAGE PREDICTION

In [45]:
dataset_groups = matches(group_matches)
dataset_groups.tail()

,Team1,Team2,Team1_FIFA_RANK,Team2_FIFA_RANK,Team1_Goalkeeper_Score,Team2_Goalkeeper_Score,Team1_Defense,Team1_Offense,Team1_Midfield,Team2_Defense,Team2_Offense,Team2_Midfield
43,Germany,Costa Rica,12,31,90.0,88.0,84.0,83.0,88.0,72.0,70.0,69.0
44,Uruguay,Ghana,13,60,80.0,74.0,81.0,84.0,80.0,76.0,76.0,78.0
45,Portugal,Korea Republic,8,29,82.0,75.0,85.0,86.0,84.0,73.0,80.0,74.0
46,Serbia,Switzerland,25,14,80.0,85.0,76.0,80.0,82.0,78.0,77.0,80.0
47,Brazil,Cameroon,1,37,89.0,67.0,85.0,86.0,86.0,77.0,78.0,75.0


In [46]:
dataset_groups_Semi = matches(group_matches_semi_Fin)
dataset_groups_Semi.tail()

,Team1,Team2,Team1_FIFA_RANK,Team2_FIFA_RANK,Team1_Goalkeeper_Score,Team2_Goalkeeper_Score,Team1_Defense,Team1_Offense,Team1_Midfield,Team2_Defense,Team2_Offense,Team2_Midfield
0,France,Morocco,3,24,87.0,82.0,84.0,88.0,87.0,81.0,82.0,76.0
1,Croatia,Argentina,16,4,82.0,84.0,78.0,77.0,84.0,82.0,89.0,84.0


In [47]:
dataset_groups_3rd = matches(group_matches_3rd)
dataset_groups_3rd.tail()




,Team1,Team2,Team1_FIFA_RANK,Team2_FIFA_RANK,Team1_Goalkeeper_Score,Team2_Goalkeeper_Score,Team1_Defense,Team1_Offense,Team1_Midfield,Team2_Defense,Team2_Offense,Team2_Midfield
0,Croatia,Morocco,16,24,82.0,82.0,78.0,77.0,84.0,81.0,82.0,76.0


In [48]:
dataset_groups_final = matches(group_matches_Fin)
dataset_groups_Semi.tail()

,Team1,Team2,Team1_FIFA_RANK,Team2_FIFA_RANK,Team1_Goalkeeper_Score,Team2_Goalkeeper_Score,Team1_Defense,Team1_Offense,Team1_Midfield,Team2_Defense,Team2_Offense,Team2_Midfield
0,France,Morocco,3,24,87.0,82.0,84.0,88.0,87.0,81.0,82.0,76.0
1,Croatia,Argentina,16,4,82.0,84.0,78.0,77.0,84.0,82.0,89.0,84.0


In [49]:
prediction_groups = gs_model.predict(dataset_groups)
proba = gs_model.predict_proba(dataset_groups)
results = print_results(dataset_groups, prediction_groups, group_matches, proba)


Qatar vs. Ecuador => Winner: Qatar
Probability of Qatar winning:  0.499
Probability of Draw:  0.283
Probability of Ecuador winning:  0.218


Senegal vs. Netherlands => Winner: Netherlands
Probability of Netherlands winning:  0.486
Probability of Draw:  0.286
Probability of Senegal winning:  0.228


England vs. IR Iran => Winner: England
Probability of England winning:  0.728
Probability of Draw:  0.173
Probability of IR Iran winning:  0.099


USA vs. Wales => Winner: USA
Probability of USA winning:  0.486
Probability of Draw:  0.259
Probability of Wales winning:  0.254


France vs. Australia => Winner: France
Probability of France winning:  0.759
Probability of Draw:  0.146
Probability of Australia winning:  0.095


Denmark vs. Tunisia => Winner: Denmark
Probability of Denmark winning:  0.589
Probability of Draw:  0.234
Probability of Tunisia winning:  0.177


Mexico vs. Poland => Winner: Poland
Probability of Poland winning:  0.393
Probability of Draw:  0.288
Probability of Mexico wi

In [50]:
prediction_groups_Semi = gs_model.predict(dataset_groups_Semi)
proba = gs_model.predict_proba(dataset_groups_Semi)
results = print_results(dataset_groups_Semi, prediction_groups_Semi, group_matches_semi_Fin, proba)


France vs. Morocco => Winner: France
Probability of France winning:  0.598
Probability of Draw:  0.233
Probability of Morocco winning:  0.169


Argentina vs. Croatia => Winner: Argentina
Probability of Croatia winning:  0.274
Probability of Draw:  0.258
Probability of Argentina winning:  0.467



AttributeError: 'DataFrame' object has no attribute 'group'

In [51]:
prediction_groups_Semi = gs_model.predict(dataset_groups_Semi)
proba = gs_model.predict_proba(dataset_groups_Semi)
results = print_results(dataset_groups_Semi, prediction_groups_Semi, group_matches_semi_Fin, proba)


France vs. Morocco => Winner: France
Probability of France winning:  0.598
Probability of Draw:  0.233
Probability of Morocco winning:  0.169


Argentina vs. Croatia => Winner: Argentina
Probability of Croatia winning:  0.274
Probability of Draw:  0.258
Probability of Argentina winning:  0.467



AttributeError: 'DataFrame' object has no attribute 'group'

In [53]:

prediction_groups_3rd = gs_model.predict(dataset_groups_3rd)
proba = gs_model.predict_proba(dataset_groups_3rd)
results = print_results(dataset_groups_3rd, prediction_groups_3rd, group_matches_3rd, proba)


Croatia vs. Morocco => Winner: Croatia
Probability of Croatia winning:  0.439
Probability of Draw:  0.297
Probability of Morocco winning:  0.264



AttributeError: 'DataFrame' object has no attribute 'group'

In [ ]:
prediction_groups_Final = gs_model.predict(dataset_groups_final)
proba = gs_model.predict_proba(dataset_groups_final)
results = print_results(dataset_groups_final, prediction_groups_Final, group_matches_Fin, proba)

In [191]:
prediction_groups_Final = gs_model.predict(dataset_groups_final)
proba = gs_model.predict_proba(dataset_groups_final)
results = print_results(dataset_groups_final, prediction_groups_Final, group_matches_Fin, proba)


Argentina vs. France => Winner: France
Probability of France winning:  0.500
Probability of Draw:  0.230
Probability of Argentina winning:  0.270



AttributeError: 'DataFrame' object has no attribute 'group'

In [48]:
team_group['points'] = 0
team_group
for i in range(results.shape[0]):
    for j in range(team_group.shape[0]):
        if results.iloc[i, 1] == team_group.iloc[j, 0]:
            team_group.iloc[j, 2] += 3

In [49]:
print(team_group.groupby(['group','team']).mean().astype(int))

                      points
group team                  
a     Ecuador              0
      Netherlands          9
      Qatar                3
      Senegal              6
b     England              9
      IR Iran              0
      USA                  6
      Wales                3
c     Argentina            9
      Mexico               3
      Poland               6
      Saudi Arabia         0
d     Australia            3
      Denmark              6
      France               9
      Tunisia              0
e     Costa Rica           0
      Germany              9
      Japan                3
      Spain                6
f     Belgium              9
      Canada               0
      Croatia              6
      Morocco              3
g     Brazil               9
      Cameroon             0
      Serbia               6
      Switzerland          3
h     Ghana                3
      Korea Republic       0
      Portugal             9
      Uruguay              6


In [50]:
round_of_16 = team_group[team_group['points'] > 5].reset_index(drop=True)
round_of_16['group'] = (4 - 1/3 * round_of_16.points).astype(int).astype(str) + round_of_16.group 
round_of_16 = round_of_16.rename(columns = {"team":"result"})
    
round_16 = winner_to_match(round_16, round_of_16)
results_round_16 = prediction_knockout(round_16)


Netherlands vs. USA => Winner: Netherlands
Probability of Netherlands winning:  0.587
Probability of Draw:  0.236
Probability of USA winning:  0.176


Argentina vs. Denmark => Winner: Argentina
Probability of Argentina winning:  0.518
Probability of Draw:  0.267
Probability of Denmark winning:  0.215


Germany vs. Croatia => Winner: Germany
Probability of Germany winning:  0.460
Probability of Draw:  0.264
Probability of Croatia winning:  0.276


Brazil vs. Uruguay => Winner: Brazil
Probability of Brazil winning:  0.645
Probability of Draw:  0.188
Probability of Uruguay winning:  0.167


England vs. Senegal => Winner: England
Probability of England winning:  0.509
Probability of Draw:  0.279
Probability of Senegal winning:  0.212


France vs. Poland => Winner: France
Probability of France winning:  0.642
Probability of Draw:  0.208
Probability of Poland winning:  0.150


Belgium vs. Spain => Winner: Spain
Probability of Spain winning:  0.482
Probability of Draw:  0.235
Probability of 

In [86]:
quarter_finals = winner_to_match(quarter_finals, results_round_16)
results_quarter_finals = prediction_knockout(quarter_finals)


final = winner_to_match(prediction_groups_Semi, dataset_groups_Semi)
results_quarter_finals = prediction_knockout(final)


Netherlands vs. Argentina => Winner: Argentina
Probability of Netherlands winning:  0.330
Probability of Draw:  0.303
Probability of Argentina winning:  0.367


Germany vs. Brazil => Winner: Germany
Probability of Germany winning:  0.397
Probability of Draw:  0.273
Probability of Brazil winning:  0.329


England vs. France => Winner: England
Probability of England winning:  0.375
Probability of Draw:  0.264
Probability of France winning:  0.361


Spain vs. Portugal => Winner: Portugal
Probability of Portugal winning:  0.390
Probability of Draw:  0.290
Probability of Spain winning:  0.320



AttributeError: 'numpy.ndarray' object has no attribute 'insert'

In [52]:
semi_finals = winner_to_match(semi_finals, results_quarter_finals)
results_finals = prediction_knockout(semi_finals)


Argentina vs. Germany => Winner: Germany
Probability of Germany winning:  0.412
Probability of Draw:  0.264
Probability of Argentina winning:  0.323


England vs. Portugal => Winner: Portugal
Probability of Portugal winning:  0.392
Probability of Draw:  0.292
Probability of England winning:  0.317



In [53]:
final = winner_to_match(final, results_finals)
# winner = prediction_knockout(final)
results_finals

,group,result
0,y1,Germany
1,y2,Portugal


In [54]:
second = results_finals[~results_finals.result.isin(winner.result)]
results_finals_3 = results_quarter_finals[~results_quarter_finals.result.isin(results_finals.result)]
results_finals_3.iloc[0, 0]='z1'
results_finals_3.iloc[1, 0]='z2'
second_final = winner_to_match(second_final, results_finals_3)
third = prediction_knockout(second_final)

NameError: name 'winner' is not defined

In [55]:
# Printing Tournament Table

In [56]:
round_16 = center_str(round_16)
quarter_finals = center_str(quarter_finals)
semi_finals = center_str(semi_finals)
final = center_str(final)
group_matches = center_str(group_matches)

In [57]:
print(round_16.iloc[0, 0]+'━━━━┓                                                                                                                             ┏━━━━'+round_16.iloc[4, 0])
print('                 ┃                                                                                                                             ┃')
print('                 ┃━━━━'+quarter_finals.iloc[0, 0]+'━━━━┓                                                                                 ┏━━━━'+quarter_finals.iloc[2, 0]+'━━━━┃')
print('                 ┃                     ┃                                                                                 ┃                     ┃')
print(round_16.iloc[0, 1]+'━━━━┛                     ┃                                                                                 ┃                     ┗━━━━'+round_16.iloc[4, 1])
print('                                       ┃━━━━'+semi_finals.iloc[0, 0]+'━━━━┓                                     ┏━━━━'+semi_finals.iloc[1, 0]+'━━━━┃')
print(round_16.iloc[1, 0]+'━━━━┓                     ┃                     ┃                                     ┃                     ┃                     ┏━━━━'+round_16.iloc[5, 0])
print('                 ┃                     ┃                     ┃                                     ┃                     ┃                     ┃')
print('                 ┃━━━━'+quarter_finals.iloc[0, 1]+'━━━━┛                     ┃                                     ┃                     ┗━━━━'+quarter_finals.iloc[2, 1]+'━━━━┃')
print('                 ┃                                           ┃                                     ┃                                           ┃')
print(round_16.iloc[1, 1]+'━━━━┛                                           ┃                                     ┃                                           ┗━━━━'+round_16.iloc[5, 1])
print('                                                             ┃━━━━'+final.iloc[0, 0]+'vs.'+final.iloc[0, 1]+'━━━━┃')
print(round_16.iloc[2, 0]+'━━━━┓                                           ┃                                     ┃                                           ┏━━━━'+round_16.iloc[6, 0])
print('                 ┃                                           ┃                                     ┃                                           ┃')
print('                 ┃━━━━'+quarter_finals.iloc[1, 0]+'━━━━┓                     ┃                                     ┃                     ┏━━━━'+quarter_finals.iloc[3, 0]+'━━━━┃')
print('                 ┃                     ┃                     ┃                                     ┃                     ┃                     ┃')
print(round_16.iloc[2, 1]+'━━━━┛                     ┃                     ┃                                     ┃                     ┃                     ┗━━━━'+round_16.iloc[6, 1])
print('                                       ┃━━━━'+semi_finals.iloc[0, 1]+'━━━━┛                                     ┗━━━━'+semi_finals.iloc[1, 1]+'━━━━┃')
print(round_16.iloc[3, 0]+'━━━━┓                     ┃                                                                                 ┃                     ┏━━━━'+round_16.iloc[7, 0])
print('                 ┃                     ┃                                                                                 ┃                     ┃')
print('                 ┃━━━━'+quarter_finals.iloc[1, 1]+'━━━━┛                                                                                 ┗━━━━'+quarter_finals.iloc[3, 1]+'━━━━┃')
print('                 ┃                                                                                                                             ┃')
print(round_16.iloc[3, 1]+'━━━━┛                                                                                                                             ┗━━━━'+round_16.iloc[7, 1])
print("                                                                 "+center2("\U0001F947"+winner.iloc[0, 1]))
print("                                                                 "+center2("\U0001F948"+second.iloc[0, 1]))
print("                                                                 "+center2("\U0001F949"+third.iloc[0, 1]))

 Netherlands ━━━━┓                                                                                                                             ┏━━━━   England   
                 ┃                                                                                                                             ┃
                 ┃━━━━ Netherlands ━━━━┓                                                                                 ┏━━━━   England   ━━━━┃
                 ┃                     ┃                                                                                 ┃                     ┃
     USA     ━━━━┛                     ┃                                                                                 ┃                     ┗━━━━   Senegal   
                                       ┃━━━━  Argentina  ━━━━┓                                     ┏━━━━   England   ━━━━┃
  Argentina  ━━━━┓                     ┃                     ┃                                     ┃                  

NameError: name 'winner' is not defined